In [53]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [54]:
# read in all the words
words = open('../names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [55]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [56]:
# shuffle up words
import random
random.seed(42)
random.shuffle(words)

In [68]:
# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


In [72]:
class Linear:

	def __init__(self, fan_in, fan_out, bias=False):
		self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5	#draw weights randomly from gaussian and normalize with sqrt(inputs)
		self.bias = torch.zeros(fan_out) if bias else None

	def __call__(self, x):
		self.out = x @ self.weight
		if self.bias is not None:
			self.out += self.bias
		return self.out

	def parameters(self):
		return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm1D:

	def __init__(self, dim, eps=1e-5, momentum=0.1):
		self.eps = eps
		self.momentum = momentum
		self.training = True	#to use bnmean during training and bnrunning for
		# parameters (trained with backprop)
		self.gamma = torch.ones(dim)
		self.beta = torch.zeros(dim)
		# buffers (trained with running momentum update)
		self.running_mean = torch.zeros(dim)
		self.running_var = torch.ones(dim)

	def __call__(self, x):
		# forward pass
		if self.training:
			if x.ndim == 2:
				dim = 0
			elif x.ndim == 3:
				dim = (0, 1)
			xmean = x.mean(dim, keepdim=True) # batch mean
			xvar = x.var(dim, keepdim=True) # batch variance
		else:
			xmean = self.running_mean
			xvar = self.running_var
		xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
		self.out = self.gamma * xhat + self.beta
		# update buffers
		if self.training:
			with torch.no_grad():
				self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
				self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
		return self.out

	def parameters(self):
		return [self.gamma, self.beta]

class Tanh:

	def __call__(self, x):
		self.out = torch.tanh(x)
		return self.out

	def parameters(self):
		return []

class Embedding:

	def __init__(self, num_embeddings, embedding_dim):
		self.weight = torch.randn((num_embeddings, embedding_dim))	#number of chars to be embedded, number of dims used

	def __call__(self, IX):
		self.out = self.weight[IX]
		return self.out

	def parameters(self):
		return [self.weight]

class FlattenConsecutive:

	def __init__(self, n):
		self.n = n

	def __call__(self, x):
		B, T, C = x.shape
		x = x.view(B, T//self.n, C*self.n)
		if x.shape[1] == 1:
			x = x.squeeze(1)
		self.out = x
		return self.out

	def parameters(self):
		return []

class Sequential:

	def __init__(self, layers):
		self.layers = layers

	def __call__(self, x):
		for layer in self.layers:
			x = layer(x)
		self.out = x
		return self.out

	def parameters(self):
		#get parameters of all layers and stretch them into one list
		return [p for layer in self.layers for p in layer.parameters()]

In [70]:
torch.manual_seed(42);

In [76]:
n_embd = 24 # dimensions of character embeddings
n_hidden = 68 # number of hidden neurons


model = Sequential([
	Embedding(vocab_size, n_embd),
	FlattenConsecutive(2), Linear(n_embd * 2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
	FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
	FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
	Linear(n_hidden, vocab_size),
])

with torch.no_grad():
	#last layer: make less confident, to stabilize loss
	model.layers[-1].weight *= 0.1

parameters = model.parameters()
print(sum(p.nelement() for p in parameters))	#number of parameters in total
for p in parameters:
	p.requires_grad = True

22370


In [86]:
e = torch.randn(2, 2, 10)
emean = e.mean((0, 1), keepdim=True)
emean.shape
print(e)
emean

tensor([[[ 0.8544,  0.8077, -0.2160,  1.4015, -0.7051,  1.3122,  0.4679,
           0.9117,  1.0501, -0.7994],
         [ 0.8423, -0.9202, -0.9165,  0.1914, -1.1408, -0.5371,  0.5894,
           0.9649,  0.6610,  0.5959]],

        [[ 1.6038,  0.7945, -1.1393,  0.8390,  1.1462,  0.0759, -0.3246,
          -0.0667, -0.3168,  1.2479],
         [-1.1180, -1.0013,  0.8578,  1.2600,  0.5922,  0.1491, -1.5011,
           0.2985, -0.2502, -1.0738]]])


tensor([[[ 0.5456, -0.0798, -0.3535,  0.9230, -0.0269,  0.2500, -0.1921,
           0.5271,  0.2860, -0.0073]]])

In [77]:
# optimization
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  logits = model(Xb)
  loss = F.cross_entropy(logits, Yb) # get loss

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  # update
  lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

  break

      0/ 200000: 3.3065


In [62]:
plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1))

RuntimeError: shape '[-1, 1000]' is invalid for input of size 1

In [65]:
#put layers into eval mode
for layer in model.layers:
	layer.training = False

In [66]:
# evaluate the loss
@torch.no_grad() # this decorator disables gradient tracking: tells pytorch to not track gradients, making it more efficient
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  logits = model(x)
  loss = F.cross_entropy(logits, y) # get loss
  print(split, loss.item())

split_loss('train')
split_loss('val')


train 3.291552782058716
val 3.292344331741333


In [67]:
# sample from the model
for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      logits = model(torch.tensor([context])) # (1,block_size,d)
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

gjoi.
tfqcwowpsbivyh.
eqikofysqazhiwvfhmcdcpzojankkujkj.
osgdfqadwhzwbsmhhk.
vmvzvpxqgttyzgpl.
dothwsahlkphlfnbqqtulyrhmp.
gqsuyxyavrqdtpq.
edvigdakriesxurznkbmgbudd.
ibbdjijuflpnbklrpcyspixqwtu.
kfcjcujjkepu.
vqp.
duebnfhkcguyilakasiqamvsuklipiubsjljn.
eumtftaapehvnboauyfliknnhi.
yofjpowq.
zrcpqchgandxhhbkcykhgsfvsqtnyuekxilyitxrumfzwqiwijonqbcjcivjd.
djedpigqixhvpmwkagj.
dodfgyjttbddxenyuetcqhwdjpp.
zeejunmzodxk.
pewybtsuddzvrzmciogygsldurxmqtercespcxtboot.
druvselofrvajfjhjgneksynrnqogudhczocmxoxtbupknotebdegifstrtwpozgyjlsbkssauaylaoabmajjugntmvxybga.
